In [36]:
import sys
import os
import numpy as np

import transforms
from bezier_interspace_transforms import tendon_disp_to_bezier_control_points, bezier_control_points_to_tendon_disp
# current_directory = os.path.dirname(os.path.realpath(__file__))
# parent_directory = os.path.dirname(current_directory)
# sys.path.append(parent_directory)
# # from bezier_set import BezierSet


class CatheterMotion(): 

    def __init__(self, p_0, l=0.2, r=0.01, n_mid_points=0, verbose=1):
        """
        Args:
            p_0 ((3,) numpy array): start point of catheter
            l (float): length of catheter
            r (float): cross section radius of catheter
            n_mid_points (int): number of middle control points
            verbose (0, 1, or 2): amount of verbosity

        Attributes:
            mode (int or None): Indicates the number of Degrees of Freedom (DoF).
            weight_matrix (numpy.ndarray): Matrix for weight calculations. Will take submatrix
                                           to use for calculations if necessary. 
            params (numpy.ndarray): Array to store catheter parameters over iterations.
            bezier_config ((1,6) numpy array): current bezier configuration (2nd and 3rd 
                                               control points respectively) of catheter

        """
        self.p_0 = np.append(p_0, 1)
        self.l = l
        self.r = r

        # self.verbose = verbose

        # if n_mid_points == 0:
        #     self.n_mid_points = 0
        # else:
        #     if n_mid_points % 2 == 0:
        #         self.n_mid_points = n_mid_points + 1
        #     else:
        #         self.n_mid_points = n_mid_points

        # self.mode = None
        # self.weight_matrix = np.zeros((3, 3))
        # self.params = np.zeros((self.n_iter + 2, 5))
        # self.bezier_config = np.zeros((6, 1))
        

    def past_frames_prediction(self, delta_u_list, p_init):
        """
        delta_u_list: start from delta_u_(n-1)
        """
        
        p_1 = np.append(p_init[:3], 1)
        p_2 = np.append(p_init[3:], 1)

        ux, uy = bezier_control_points_to_tendon_disp(self.p_0, p_1, p_2, self.l, self.r)
        print("Initial ux, uy: ", ux, uy)

        delta_u_cumulative = np.array([ux, uy])
        bezier_control_points = []

        for delta_u in delta_u_list:
            delta_u_cumulative -= delta_u
            p_1_new, p_2_new = tendon_disp_to_bezier_control_points(delta_u_cumulative[0], delta_u_cumulative[1], self.l, self.r, self.p_0)
            print('ux, uy', delta_u_cumulative[0], delta_u_cumulative[1])
            # bezier_points = np.array([p_1_new[:-1], p_2_new[:-1]])
            bezier_points = np.concatenate((p_1_new[:-1], p_2_new[:-1]))
            bezier_control_points.append(bezier_points)

        return np.array(bezier_control_points)


In [37]:
def read_gt_params(cc_specs_path):
    """
    The order of gt: [p_start, p_end, c1, c2]
    """
    
    para_gt_np = np.load(cc_specs_path)
    matrix = np.squeeze(para_gt_np)
    c1 = matrix[2]
    c2 = matrix[3]
    p_start = matrix[0]
    p_end = matrix[1]
    p_mid = 3/4 * (c1 + p_end/3)
    p1 = 2*p_mid - 0.5*p_start - 0.5*p_end
    result_vector = np.concatenate((p1, p_end))
    return result_vector 

In [38]:
case_naming = 'E:/OneDrive - UC San Diego/UCSD/Lab/Catheter/diff_catheter/scripts/test_diff_render_catheter_v2/gt_dataset3/gt_6_0.0006_-0.0010_0.2_0.01'
# case_naming = 'E:/OneDrive - UC San Diego/UCSD/Lab/Catheter/diff_catheter/scripts/test_diff_render_catheter_v2/blender_imgs/test_gt_4'
img_save_path = case_naming + '.png'
cc_specs_path = case_naming + '.npy'

para_gt_np = read_gt_params(cc_specs_path)
print(para_gt_np)

[ 2.15634587e-02 -6.05764476e-04  5.16317712e-01  1.65068886e-01
 -2.39781477e-01  9.49010349e-01]


In [39]:
# Create an instance of CatheterMotion
p_0 = np.array([2e-2, 2e-3, 0])
l = 0.2
r = 0.01
n_mid_points = 0                 
catheter = CatheterMotion(p_0, l, r)

# Initial Bezier control points (6D vector; first 3 for p1, last 3 for p2)
p_init = np.array([0.034, -0.003, 0.526, 0.13, -0.24, 0.6],
                     dtype=np.float32)

# List of changes in control parameters (delta_u_list)
delta_u_list = [[0, -0.0002], [0, -0.0002], [0, -0.0002], [0, -0.0002]]

# Run the test
predicted_bezier_points = catheter.past_frames_prediction(delta_u_list, p_init)

# Print the results
print("Predicted Bezier Control Points for Past Frames:")
print(predicted_bezier_points)
print(predicted_bezier_points.shape)



Initial ux, uy:  0.0021725185137369146 -0.005460919249087846
ux, uy 0.0021725185137369146 -0.005260919249087846
ux, uy 0.0021725185137369146 -0.005060919249087846
ux, uy 0.0021725185137369146 -0.004860919249087847
ux, uy 0.0021725185137369146 -0.004660919249087847
Predicted Bezier Control Points for Past Frames:
[[ 0.11591201 -0.23025824  0.65450297  0.27809585 -0.62299879  0.10407099]
 [ 0.11180137 -0.21185287  0.65473224  0.29125585 -0.62989517  0.1396225 ]
 [ 0.10762768 -0.19406327  0.653998    0.30438702 -0.63430405  0.17619358]
 [ 0.10341104 -0.17694998  0.65236998  0.31744453 -0.63613722  0.2135932 ]]
(4, 6)


In [40]:
p1, p2 = tendon_disp_to_bezier_control_points(0.0021725185137369146, -0.005460919249087846, l, r, np.array([2e-2, 2e-3, 0, 1]))
print(p1, p2)

[ 0.11993993 -0.24921253  0.65324587  1.        ] [ 0.26495113 -0.6137178   0.06972045  1.        ]


In [41]:
p_0 = np.array([2e-2, 2e-3, 0])
p_0_h = np.array([2e-2, 2e-3, 0, 1])
l = 0.2
r = 0.01
ux = 0.002
uy = 0.003

In [42]:
def cc_transform_3dof(p_0, ux, uy, l, r, s=1):
    """
    Calculate constant curvature 3DoF transformation
    
    Args:
        p_0 ((3,) numpy array): start point of catheter
        ux (float): 1st pair of tendon length (responsible for catheter bending)
        uy (float): 2nd pair of tendon length (responsible for catheter bending)
        l (float): length of catheter
        r (float): cross section radius of catheter
        s (float from 0 to 1 inclusive): s value representing position on the CC curve
    """
    p_0_4d = np.append(p_0, 1)
    u = np.sqrt(ux ** 2 + uy ** 2)
    k = u / (l * r)

    T = np.array([
        [1 + (ux ** 2) / (u ** 2) * (np.cos(k * s) - 1), ux * uy / (u ** 2) * (np.cos(k * s) - 1),       -1 * ux / u * np.sin(k * s), r * l * ux * (1 - np.cos(k * s)) / (u ** 2)],
        [ux * uy / (u ** 2) * (np.cos(k * s) - 1),       1 + (uy ** 2) / (u ** 2) * (np.cos(k * s) - 1), -1 * uy / u * np.sin(k * s), r * l * uy * (1 - np.cos(k * s)) / (u ** 2)],
        [ux / u * np.sin(k * s),                         uy / u * np.sin(k * s),                         np.cos(k * s),               r * l * np.sin(k * s) / u                  ],
        [0,                                              0,                                              0,                           1                                          ]], dtype=object)

    return (T @ p_0_4d)[:3]

p_end = cc_transform_3dof(p_0, ux, uy, l, r)
p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h)
print(p_end)
print(p1, p2)


[0.3897281434379607 0.5565922151569411 0.5522579368980701]
[0.06328963 0.06693444 0.61385996 1.        ] [0.38972814 0.55659222 0.55225794 1.        ]


In [43]:
p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h)
ux_solve, uy_solve = bezier_control_points_to_tendon_disp(p_0_h, p1, p2, l, r)

print(ux_solve, uy_solve)

0.0020457509260103923 0.003068626389015589
